
#  Interactive Dashboard Project (Google Colab)

## Project Overview
This project is an interactive dashboard created in **Google Colab** using **Python**.  
It visualizes the dataset **[Bike store sales data]** and provides actionable insights to help understand patterns, trends, and key metrics.

**Key Features:**
- Interactive visualizations within the notebook

- Easy-to-understand charts and graphs
- Filters and selections to explore data dynamically
- Key insights highlighted for decision-making

---

## Dataset
- **Source:** [Provide dataset source or attach dataset in the `data/`'/content/Sales.csv.xlsx']  


pip install notebook


In [ ]:
!pip install dash plotly pandas

import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# -------------------------------
# 1. Load Dataset
# -------------------------------
df = pd.read_excel("/content/Sales.csv.xlsx")

# Ensure Date is in datetime format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Create Month-Year column for trend analysis
df['Month'] = df['Date'].dt.to_period("M").astype(str)

# Use "State" as store/location equivalent
df.rename(columns={"State": "Location"}, inplace=True)

# -------------------------------
# 2. Initialize the App
# -------------------------------
app = dash.Dash(__name__)
app.title = "Bike Store Dashboard"

# -------------------------------
# 3. App Layout
# -------------------------------
app.layout = html.Div([
    html.H1("🚴 Bike Store Sales Dashboard", style={'textAlign': 'center'}),

    # Filters
    html.Div([
        html.Div([
            html.Label("Select Country"),
            dcc.Dropdown(
                id='country-filter',
                options=[{'label': c, 'value': c} for c in df['Country'].dropna().unique()],
                value=[],
                multi=True,
                placeholder="Filter by country"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.Label("Select Product Category"),
            dcc.Dropdown(
                id='category-filter',
                options=[{'label': c, 'value': c} for c in df['Product_Category'].dropna().unique()],
                value=[],
                multi=True,
                placeholder="Filter by category"
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.Label("Select Date Range"),
            dcc.DatePickerRange(
                id='date-filter',
                min_date_allowed=df['Date'].min(),
                max_date_allowed=df['Date'].max(),
                start_date=df['Date'].min(),
                end_date=df['Date'].max()
            )
        ], style={'width': '30%', 'display': 'inline-block', 'padding': '10px'}),
    ]),

    html.Hr(),

    # KPI Cards
    html.Div([
        html.Div(id='total-revenue', style={'width': '30%', 'display': 'inline-block', 'textAlign': 'center'}),
        html.Div(id='total-orders', style={'width': '30%', 'display': 'inline-block', 'textAlign': 'center'}),
        html.Div(id='avg-order-value', style={'width': '30%', 'display': 'inline-block', 'textAlign': 'center'}),
    ], style={'margin': '20px 0'}),

    html.Hr(),

    # Graphs
    html.Div([
        dcc.Graph(id='sales-trend', style={'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(id='top-products', style={'width': '50%', 'display': 'inline-block'}),
    ]),

    html.Div([
        dcc.Graph(id='revenue-by-country', style={'width': '50%', 'display': 'inline-block'}),
        dcc.Graph(id='revenue-by-category', style={'width': '50%', 'display': 'inline-block'}),
    ])
])

# -------------------------------
# 4. Callbacks
# -------------------------------
@app.callback(
    [
        Output('total-revenue', 'children'),
        Output('total-orders', 'children'),
        Output('avg-order-value', 'children'),
        Output('sales-trend', 'figure'),
        Output('top-products', 'figure'),
        Output('revenue-by-country', 'figure'),
        Output('revenue-by-category', 'figure')
    ],
    [
        Input('country-filter', 'value'),
        Input('category-filter', 'value'),
        Input('date-filter', 'start_date'),
        Input('date-filter', 'end_date')
    ]
)
def update_dashboard(selected_countries, selected_categories, start_date, end_date):
    # Filter by date
    filtered_df = df[
        (df['Date'] >= pd.to_datetime(start_date)) &
        (df['Date'] <= pd.to_datetime(end_date))
    ]

    # Filter by country
    if selected_countries:
        filtered_df = filtered_df[filtered_df['Country'].isin(selected_countries)]

    # Filter by category
    if selected_categories:
        filtered_df = filtered_df[filtered_df['Product_Category'].isin(selected_categories)]

    # KPIs
    total_revenue = filtered_df['Revenue'].sum()
    total_orders = filtered_df['Order_Quantity'].sum()
    avg_order_value = total_revenue / total_orders if total_orders > 0 else 0

    kpi_style = {'fontSize': '20px', 'fontWeight': 'bold'}
    kpi1 = html.Div([html.H3("Total Revenue"), html.P(f"${total_revenue:,.2f}", style=kpi_style)])
    kpi2 = html.Div([html.H3("Total Units Sold"), html.P(f"{total_orders}", style=kpi_style)])
    kpi3 = html.Div([html.H3("Avg Revenue per Unit"), html.P(f"${avg_order_value:,.2f}", style=kpi_style)])

    # Sales trend over time
    monthly_sales = filtered_df.groupby('Month').agg({'Revenue': 'sum'}).reset_index()
    sales_trend = px.line(monthly_sales, x='Month', y='Revenue', title='Sales Trend Over Time')

    # Top product categories
    product_sales = filtered_df.groupby('Product_Category').agg({'Revenue': 'sum'}).reset_index()
    product_sales = product_sales.sort_values(by='Revenue', ascending=False)
    top_products = px.bar(product_sales, x='Product_Category', y='Revenue', title='Top Product Categories by Revenue')

    # Revenue by country
    country_sales = filtered_df.groupby('Country').agg({'Revenue': 'sum'}).reset_index()
    country_chart = px.bar(country_sales, x='Country', y='Revenue', title='Revenue by Country')

    # Revenue share by category
    category_chart = px.pie(filtered_df, names='Product_Category', values='Revenue', title='Revenue Share by Product Category')

    return kpi1, kpi2, kpi3, sales_trend, top_products, country_chart, category_chart

# -------------------------------
# 5. Run the App
# -------------------------------
if __name__ == '__main__':
    app.run(debug=True)



<IPython.core.display.Javascript object>

Actionable Insights from Bike Sales Dashboard
 Overall KPIs:

Total Revenue: $85.3M

Total Profit: $32.2M

Total Units Sold: 1.35M

Average Revenue per Unit: $63.38

Top Product Categories:

Bikes: $61.8M revenue (dominates sales)

Accessories: $15.1M revenue

Clothing: $8.4M revenue

Bikes generate ~72% of revenue → focus marketing & inventory on bikes.

Top Countries by Revenue:

United States: $27.9M

Australia: $21.3M

United Kingdom: $10.6M
USA & Australia are the strongest markets; consider targeted growth in the UK.

 Sales Trend:

Revenue shows consistent demand with seasonal fluctuations.
Stock management & promotions should anticipate seasonal peaks.

* Customer Demographics:

Gender Split: Female (49%), Male (51%) → Balanced customer base.

Age Groups:

Adults (35–64): $42.6M

Young Adults (25–34): $30.7M

Youth (<25): $11.7M

Seniors (64+): negligible
 Adults & young adults are the core buyers → target campaigns to 25–64 age group.

* Business Recommendations:

Prioritize Bike sales campaigns (main revenue driver).

Upsell accessories & clothing with bike purchases.

Expand marketing in Australia & UK (biggest growth potential).

Focus advertising on 25–64 age group, with gender-neutral promotions.

Leverage seasonal trends (offer discounts in low months to stabilize revenue).